# 防止过拟合方法---dropout


In [31]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [46]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])


### 定义keep_prob参数

In [47]:
keep_prob = tf.placeholder(tf.float32)

In [48]:
#第一层
W_h1 = tf.Variable(tf.truncated_normal([784, 512], mean=0., stddev=0.1))
b_h1 = tf.Variable(tf.zeros([512]))
h_linear_output1 = tf.matmul(x, W_h1) + b_h1
h_output1 = tf.nn.tanh(h_linear_output1)

In [49]:

#第二层
W_h = tf.Variable(tf.truncated_normal([512, 256], mean=0., stddev=0.1))
b_h = tf.Variable(tf.zeros([256]))
h_linear_output2 = tf.matmul(h_output1, W_h) + b_h
h_output2 = tf.nn.tanh(h_linear_output2)
#dropout层
h_fc_drop1 = tf.nn.dropout(h_output2, keep_prob)


In [53]:

#第三层
W_h = tf.Variable(tf.truncated_normal([256, 128], mean=0., stddev=0.1))
b_h = tf.Variable(tf.zeros([128]))
h_linear_output3 = tf.matmul(h_fc_drop1, W_h) + b_h
h_output3 = tf.nn.tanh(h_linear_output3)
#dropout层
h_fc_drop2 = tf.nn.dropout(h_output3, keep_prob)

In [54]:
W = tf.Variable(tf.truncated_normal([128, 10], mean=0., stddev=0.1))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(h_fc_drop2, W) + b

In [55]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [56]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [59]:
softmax_result = tf.nn.softmax(y)
pred = tf.argmax(softmax_result, axis=1)
true = tf.argmax(y_, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, true), tf.float32))

### keep_prob 设为1时的训练效果（即训练时不添加dropout）：

In [58]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(5000):
    batch = mnist.train.next_batch(128)
    _, loss_, acu = sess.run([train_step, loss, accuracy], {x: batch[0], y_: batch[1],keep_prob:1})
    if i % 100 == 0:
        print("Step:{:>5}, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(i, loss_, acu))
loss_, acu = sess.run([loss, accuracy], {x: mnist.test.images, 
                                                        y_: mnist.test.labels,keep_prob:1})
print("Training Finished, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(loss_, acu))

Step:    0, Loss: 2.3768, Accuracy: 10.94%
Step:  100, Loss: 0.2586, Accuracy: 89.06%
Step:  200, Loss: 0.2087, Accuracy: 92.97%
Step:  300, Loss: 0.2284, Accuracy: 93.75%
Step:  400, Loss: 0.1337, Accuracy: 94.53%
Step:  500, Loss: 0.1063, Accuracy: 95.31%
Step:  600, Loss: 0.0785, Accuracy: 98.44%
Step:  700, Loss: 0.1048, Accuracy: 96.88%
Step:  800, Loss: 0.1048, Accuracy: 98.44%
Step:  900, Loss: 0.0493, Accuracy: 96.88%
Step: 1000, Loss: 0.0659, Accuracy: 97.66%
Step: 1100, Loss: 0.0483, Accuracy: 99.22%
Step: 1200, Loss: 0.1438, Accuracy: 96.09%
Step: 1300, Loss: 0.0704, Accuracy: 97.66%
Step: 1400, Loss: 0.0436, Accuracy: 98.44%
Step: 1500, Loss: 0.0270, Accuracy: 99.22%
Step: 1600, Loss: 0.0236, Accuracy: 99.22%
Step: 1700, Loss: 0.0355, Accuracy: 99.22%
Step: 1800, Loss: 0.1021, Accuracy: 98.44%
Step: 1900, Loss: 0.0637, Accuracy: 97.66%
Step: 2000, Loss: 0.0421, Accuracy: 99.22%
Step: 2100, Loss: 0.0311, Accuracy: 99.22%
Step: 2200, Loss: 0.0188, Accuracy: 99.22%
Step: 2300,

### 添加dropout 的训练：

In [60]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(5000):
    batch = mnist.train.next_batch(128)
    _, loss_, acu = sess.run([train_step, loss, accuracy], {x: batch[0], y_: batch[1],keep_prob:0.8})
    if i % 100 == 0:
        print("Step:{:>5}, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(i, loss_, acu))
loss_, acu = sess.run([loss, accuracy], {x: mnist.test.images, 
                                                        y_: mnist.test.labels,keep_prob:1})
print("Training Finished, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(loss_, acu))

Step:    0, Loss: 2.4576, Accuracy:  8.59%
Step:  100, Loss: 0.1826, Accuracy: 94.53%
Step:  200, Loss: 0.2070, Accuracy: 92.19%
Step:  300, Loss: 0.2159, Accuracy: 92.19%
Step:  400, Loss: 0.2363, Accuracy: 94.53%
Step:  500, Loss: 0.1746, Accuracy: 93.75%
Step:  600, Loss: 0.1212, Accuracy: 96.09%
Step:  700, Loss: 0.2347, Accuracy: 91.41%
Step:  800, Loss: 0.2745, Accuracy: 92.97%
Step:  900, Loss: 0.2629, Accuracy: 93.75%
Step: 1000, Loss: 0.1604, Accuracy: 95.31%
Step: 1100, Loss: 0.1082, Accuracy: 96.09%
Step: 1200, Loss: 0.1129, Accuracy: 96.09%
Step: 1300, Loss: 0.0900, Accuracy: 96.09%
Step: 1400, Loss: 0.1682, Accuracy: 95.31%
Step: 1500, Loss: 0.2857, Accuracy: 92.97%
Step: 1600, Loss: 0.1641, Accuracy: 95.31%
Step: 1700, Loss: 0.0474, Accuracy: 98.44%
Step: 1800, Loss: 0.0967, Accuracy: 98.44%
Step: 1900, Loss: 0.0449, Accuracy: 99.22%
Step: 2000, Loss: 0.0597, Accuracy: 99.22%
Step: 2100, Loss: 0.0960, Accuracy: 95.31%
Step: 2200, Loss: 0.1097, Accuracy: 95.31%
Step: 2300,